<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/Lab_04_Team1/Lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [999 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 https

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Movies-Activity").getOrCreate()

df_movies = spark.read \
            .option("inferSchema", "true") \
            .json("/content/drive/MyDrive/movies.json")

df_movies.printSchema()
df_movies.show(n=7)


#Calculate Profit Margin: Compute the profit margin for each movie
#by subtracting the Production Budget from the Worldwide Gross
#and then dividing by the Production Budget
df_ProfitMargin = df_movies.withColumn('Profit Margin', ((df_movies['Worldwide_Gross'] - df_movies['Production_Budget']) / df_movies['Production_Budget']))
df_ProfitMargin.show(n=10)

#Determine if a Movie is a Box Office Hit: Create a column
#Box Office Hit that categorizes movies as ”Hit” if
#Worldwide Gross is greater than twice the Production Budget,
#and ”Flop” otherwise.
df_BoxOfficeHit = df_movies.withColumn('Box_Office_hit?',
                                       when(df_movies['Worldwide_Gross'] > (2 * df_movies['Production_Budget']), "Hit").otherwise("Flop"))
df_BoxOfficeHit.show(n=10)

#Convert Release Date to a Standard Format: Convert the
#Release Date from DD-MMM-YY to a DateType column and
#extract the year in a new column.
df_date = df_movies.withColumn(
    "Release_Date",
    when(df_movies["Release_Date"].rlike("^\d{1,2}-[A-Za-z]{3}-\d{2}$"), to_date(df_movies["Release_Date"], "d-MMM-yy"))
    .when(df_movies["Release_Date"].rlike("^\d{4}-\d{2}-\d{2}$"), to_date(df_movies["Release_Date"], "yyyy-MM-dd"))
    .when(df_movies["Release_Date"].rlike("^[A-Za-z]+, \d{4}$"), to_date(df_movies["Release_Date"], "MMMM, yyyy"))
    .otherwise(None)
)
df_date = df_date.withColumn("Release_Year", year(df_date["Release_Date"]))


df_date.select("Release_Date", "Release_Year").show(n=10)

#Create a new column: IMDB Rating Category to categorize movies
#based on their IMDB Rating: ”High” if the rating is 7.0 or above,
#”Medium” if between 5.0 and 6.9, and ”Low” if below 5.0.
df_IMDB = df_movies.withColumn('IMDB_Simple_Rating',
                               when(df_movies['IMDB_Rating'] >= 7.0 , "High").when(df_movies['IMDB_Rating'] >= 5.0 , "Medium").otherwise("Low"))
df_IMDB.show(n=10)

#Calculate Average IMDB Rating for Each Distributor: Calculate the
#average IMDB Rating for each Distributor and create a new
#DataFrame with this information.
df_Distibutor = df_movies.groupby('Distributor').agg(avg(col('IMDB_Rating')).alias('IMDB_AVG_Rating'))
df_Distibutor.show(n=10)

spark.stop()


root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)

+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|       Creative_Type|         Director|Distributor|IMDB_R